In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


In [0]:
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))

@tf.function
def preprocess(x, y):
  x = tf.expand_dims(x, 2)
  x = tf.cast(x, tf.float32)
  y = tf.cast(y, tf.float32)
  return x, y

train_data = train_data.map(preprocess)
train_data = train_data.batch(128)
train_data = train_data.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
val_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
val_data = val_data.map(preprocess)
val_data = val_data.batch(128)
val_data = val_data.prefetch(tf.data.experimental.AUTOTUNE)

In [6]:
train_data

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.float32)>

In [7]:
val_data

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.float32)>

In [0]:
class Residual_Block(tf.keras.layers.Layer):
  def __init__(self, units):
    super(Residual_Block, self).__init__()
    self.units = units
    self.l1 = tf.keras.layers.Conv2D(units, 3, 1, 'same', activation='relu')
    self.l2 = tf.keras.layers.Conv2D(units, 3, 1, 'same', activation='relu')
    self.l3 = tf.keras.layers.Conv2D(units, 3, 1, 'same', activation='relu')
  
  def call(self, inputs):
    reserve = inputs
    x = self.l1(inputs)
    x = self.l2(x)
    x = self.l3(x)
    out = x + reserve
    return out


In [0]:
class Simple_Model(tf.keras.Model):
  def __init__(self, units):
    super(Simple_Model, self).__init__()
    self.units = units
    self.l1 = tf.keras.layers.Conv2D(units, 7, 1, 'same', activation='relu')
    self.r1 = Residual_Block(self.units)
    self.r2 = Residual_Block(self.units)
    self.gb = tf.keras.layers.GlobalMaxPool2D()
    self.d1 = tf.keras.layers.Dense(units, activation='relu')
    self.d2 = tf.keras.layers.Dense(10, activation='softmax')

  def call(self, inputs):
    x = self.l1(inputs)
    x = self.r1(x)
    x = tf.keras.layers.MaxPool2D()(x)
    x = self.r2(x)
    x = self.gb(x)
    x = self.d1(x)
    x = self.d2(x)
    return x

In [0]:
model_1 = Simple_Model(128)

Keras Check Point

In [0]:
loss = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer= tf.keras.optimizers.Adam()

model_1.compile(loss=loss, optimizer=optimizer, metrics=['acc'])

In [0]:
ckpt_callback = tf.keras.callbacks.ModelCheckpoint('/content/hist_1', verbose=1, save_best_only=True, save_weights_only=True)

hist_1 = model_1.fit(train_data, validation_data=val_data, epochs=50, callbacks=[ckpt_callback])

Train for 469 steps, validate for 79 steps
Epoch 1/50
468/469 [============================>.] - ETA: 0s - loss: 1.5059 - acc: 0.8491
Epoch 00001: val_loss improved from inf to 0.27029, saving model to /content/hist_1
469/469 [==============================] - 39s 84ms/step - loss: 1.5034 - acc: 0.8492 - val_loss: 0.2703 - val_acc: 0.9276
Epoch 2/50
468/469 [============================>.] - ETA: 0s - loss: 0.1374 - acc: 0.9642
Epoch 00002: val_loss did not improve from 0.27029
469/469 [==============================] - 39s 83ms/step - loss: 0.1380 - acc: 0.9641 - val_loss: 0.3198 - val_acc: 0.9364
Epoch 3/50
468/469 [============================>.] - ETA: 0s - loss: 0.1156 - acc: 0.9718
Epoch 00003: val_loss did not improve from 0.27029
469/469 [==============================] - 39s 82ms/step - loss: 0.1162 - acc: 0.9717 - val_loss: 0.3763 - val_acc: 0.9373
Epoch 4/50
468/469 [============================>.] - ETA: 0s - loss: 0.1060 - acc: 0.9750
Epoch 00004: val_loss did not improve 

In [0]:
tf.random.set_seed(1228)

In [0]:
model_1.evaluate(val_data)

79/79 [==============================] - 2s 30ms/step - loss: 0.0663 - acc: 0.9908


[0.06627106703183422, 0.9908]

In [0]:
model_reload = Simple_Model(128)

In [0]:
latest = tf.train.latest_checkpoint('/content/ckpt')

latest
model_reload.load_weights(latest)

In [0]:
model_reload.compile(loss=loss, optimizer=optimizer, metrics=['acc'])

In [0]:
model_reload.evaluate(val_data)

79/79 [==============================] - 2s 31ms/step - loss: 0.0379 - acc: 0.9919


[0.0378549385956756, 0.9919]

## Use Checkpoint

In [0]:
# !mkdir use_ckpt

ckpt_model = Simple_Model(512)
ckpt_model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])

checkpoint_dir = '/content/use_ckpt'
callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir)

ckpt_model.fit(train_data, epochs=10, validation_data=val_data, callbacks=[callback])

Train for 469 steps, validate for 79 steps
Epoch 1/10
469/469 [==============================] - 261s 556ms/step - loss: 1148.8729 - acc: 0.7251 - val_loss: 0.4145 - val_acc: 0.8722
Epoch 2/10
469/469 [==============================] - 260s 554ms/step - loss: 0.1939 - acc: 0.9418 - val_loss: 0.1793 - val_acc: 0.9442
Epoch 3/10
469/469 [==============================] - 260s 554ms/step - loss: 0.1265 - acc: 0.9615 - val_loss: 0.1011 - val_acc: 0.9682
Epoch 4/10
469/469 [==============================] - 260s 554ms/step - loss: 0.0994 - acc: 0.9702 - val_loss: 0.0911 - val_acc: 0.9714
Epoch 5/10
469/469 [==============================] - 260s 554ms/step - loss: 0.0852 - acc: 0.9741 - val_loss: 0.0980 - val_acc: 0.9690
Epoch 6/10
469/469 [==============================] - 260s 554ms/step - loss: 0.0766 - acc: 0.9769 - val_loss: 0.0903 - val_acc: 0.9711
Epoch 7/10
469/469 [==============================] - 260s 554ms/step - loss: 0.0678 - acc: 0.9791 - val_loss: 0.0918 - val_acc: 0.9711
Ep

In [0]:
restore_model = Simple_Model(512)

In [0]:
restore_model.compile(loss=loss, optimizer=optimizer, metrics=['acc'])

In [0]:
restore_model.evaluate(val_data)

79/79 [==============================] - 15s 193ms/step - loss: 20.5937 - acc: 0.0985


[20.593723804135866, 0.0985]

In [0]:
tf.train.latest_checkpoint('')

'use_ckpt'

In [0]:
restore_model.load_weights(tf.train.latest_checkpoint(''))

In [0]:
restore_model.evaluate(val_data)

79/79 [==============================] - 14s 183ms/step - loss: 0.1367 - acc: 0.9622


[0.1367177240774507, 0.9622]

In [0]:
# Save the weights
ckpt_model.save_weights('model_weights.h5')

In [0]:
restore_model2 = Simple_Model(512)

In [0]:
tf.saved_model.save(ckpt_model, 'test')

INFO:tensorflow:Assets written to: test/assets


In [0]:
mod = tf.keras.models.load_model('test')

In [0]:
mod

In [0]:
mod.evaluate(val_data)

79/79 [==============================] - 14s 183ms/step - loss: 0.1363 - acc: 0.9622


[0.13629496727681048, 0.9622]

## CKPT with tf function

In [0]:
model = Simple_Model(128)

In [0]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
my_opt = tf.keras.optimizers.Adam(0.0008)

acc_metrics = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')

@tf.function
def train_step(img, label):
  with tf.GradientTape() as tape:
    pred = model(img)
    loss = loss_object(label, pred)
  grad = tape.gradient(loss, model.trainable_variables)
  my_opt.apply_gradients(zip(grad, model.trainable_variables))

  acc_metrics(label, pred)
  return loss

In [0]:
checkpoint = tf.train.Checkpoint(model = model, optimizer=my_opt)
manager = tf.train.CheckpointManager(checkpoint, directory='/content', max_to_keep=3)

In [21]:
EPOCHS = 10

for epoch in range(EPOCHS):
  acc_metrics.reset_states()

  for img, label in train_data:
    loss = train_step(img, label)
  print(f'Epoch: {epoch}\t Acc: {acc_metrics.result()}')
  manager.save()


Epoch: 0	 Acc: 0.8545500040054321
Epoch: 1	 Acc: 0.9609333276748657
Epoch: 2	 Acc: 0.972083330154419
Epoch: 3	 Acc: 0.9745833277702332
Epoch: 4	 Acc: 0.9801999926567078
Epoch: 5	 Acc: 0.9824166893959045
Epoch: 6	 Acc: 0.9845666885375977
Epoch: 7	 Acc: 0.9820500016212463
Epoch: 8	 Acc: 0.9846500158309937
Epoch: 9	 Acc: 0.9862833619117737


In [0]:
val_acc = tf.keras.metrics.SparseCategoricalAccuracy()

@tf.function
def val_step(img, label):
  pred = model(img)
  val_acc(label, img)

In [29]:
for img, label in val_data:
  val_logit = model(img)
  val_acc(label, val_logit)

print(val_acc.result())

tf.Tensor(0.9736, shape=(), dtype=float32)


In [44]:
# Load

model = Simple_Model(128)

for img, label in val_data:
  val_logit = model(img)
  val_acc(label, val_logit)

print(val_acc.result())

checkpoint = tf.train.Checkpoint(model = model, optimizer=my_opt)
manager = tf.train.CheckpointManager(checkpoint, directory='/content', max_to_keep=3)

tf.Tensor(0.54445, shape=(), dtype=float32)


In [45]:
checkpoint.restore(manager.latest_checkpoint)

In [46]:
val_acc.reset_states()
for img, label in val_data:
  val_logit = model(img)
  val_acc(label, val_logit)

print(val_acc.result())

tf.Tensor(0.9736, shape=(), dtype=float32)


In [43]:
val_acc.